In [1]:
import math
import numpy as np
import os
import pandas as pd
import pickle
from PIL import Image
from sklearn.metrics import confusion_matrix, matthews_corrcoef
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer, GPT2Model, pipeline, AutoModelForCausalLM, \
AutoTokenizer, BitsAndBytesConfig, LlamaTokenizer, LlamaModel, AutoModelForTextEncoding
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from transformers import RobertaModel, RobertaTokenizer


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [8]:
human_bert = np.load('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/code.old/Deepfake_dataset/Squad/BERT/human_embeddings.npy', allow_pickle=True)
human_gpt = np.load('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/code.old/Deepfake_dataset/Squad/Llama/human_embeddings.npy', allow_pickle=True)
human_flan = np.load('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/code.old/Deepfake_dataset/Squad/FLAN/human_embeddings.npy', allow_pickle=True)


ai_bert = np.load('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/code.old/Deepfake_dataset/Squad/BERT/gpt_embeddings.npy', allow_pickle=True)
ai_gpt = np.load('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/code.old/Deepfake_dataset/Squad/Llama/gpt_embeddings.npy', allow_pickle=True)
ai_flan = np.load('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/code.old/Deepfake_dataset/Squad/FLAN/gpt_embeddings.npy', allow_pickle=True)


bert_min_length = min(len(human_bert), len(ai_bert))
human_bert = human_bert[:bert_min_length]
ai_bert = ai_bert[:bert_min_length]

gpt_min_length = min(len(human_gpt), len(ai_gpt))
human_gpt = human_gpt[:bert_min_length]
ai_gpt = ai_gpt[:bert_min_length]

flan_min_length = min(len(human_flan), len(ai_flan))
human_flan = human_flan[:bert_min_length]
ai_flan = ai_flan[:bert_min_length]

ai_embed_length = len(ai_gpt[0])
print(ai_embed_length, len(ai_bert[0][0]))
print(len(human_bert), len(human_gpt), len(human_flan), len(ai_bert), len(ai_gpt), len(ai_flan))

human_bert = list(human_bert)
ai_bert = list(ai_bert)
human_gpt = list(human_gpt)
ai_gpt = list(ai_gpt)
human_flan = list(human_flan)
ai_flan = list(ai_flan)

human_bert = torch.tensor(human_bert)
ai_bert = torch.tensor(ai_bert)
human_gpt = torch.tensor(human_gpt).reshape([bert_min_length, 1, 5120])
ai_gpt = torch.tensor(ai_gpt).reshape([bert_min_length, 1, 5120])
human_flan = torch.tensor(human_flan).reshape([bert_min_length, 1, 4096])
ai_flan = torch.tensor(ai_flan).reshape([bert_min_length, 1, 4096])


human_bert = human_bert.view(bert_min_length, 16,64)
human_gpt = human_gpt.view(bert_min_length, 80,64)
human_flan = human_flan.view(bert_min_length, 64,64)

ai_bert = ai_bert.view(bert_min_length, 16,64)
ai_gpt = ai_gpt.view(bert_min_length, 80,64)
ai_flan = ai_flan.view(bert_min_length, 64,64)

human = torch.cat([human_bert, human_gpt, human_flan], dim=1)
ai = torch.cat([ai_bert, ai_gpt, ai_flan], dim=1)
embeds = torch.cat((human, ai), dim=0)
zeros_tensor = torch.zeros((bert_min_length))
ones_tensor = torch.ones((bert_min_length))

# Concatenate the tensors along the first dimension
labels = torch.cat((zeros_tensor, ones_tensor), dim=0)

# Display the result tensor
print(labels)

5120 1024
2278 2278 2278 2278 2278 2278
tensor([0., 0., 0.,  ..., 1., 1., 1.])


In [9]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

test_size = 0.2
dataset = TensorDataset(embeds, labels)

# Split dataset into train and test sets
train_dataset, test_dataset = train_test_split(dataset, test_size=test_size, random_state=42)

# Define batch size
batch_size = 32

# Create DataLoader for training set
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
import torch

import torch.nn as nn
import torch.optim as optim




In [10]:
class GPT_CNN2D(nn.Module):
    def __init__(self, embedding_model):
        super(GPT_CNN2D, self).__init__()
        self.em_model = embedding_model
        if embedding_model == "gpt":
            self.conv1 = nn.Conv2d(1, 64, kernel_size=5, padding=1)
            self.pool = nn.MaxPool2d(kernel_size=2)
            self.conv2 = nn.Conv2d(64, 32, kernel_size=3, padding=1)
            self.pool = nn.MaxPool2d(kernel_size=2)
            self.flatten = nn.Flatten()
            self.fc1 = nn.Linear(1056, 128)
            self.fc2 = nn.Linear(128, 2)
            self.relu = nn.ReLU()
            # self.sigmoid = nn.Sigmoid()
        elif embedding_model == "mistral" or embedding_model == "flan":
            self.conv1 = nn.Conv2d(1, 32, kernel_size=8, padding=1, stride = 2)
            self.pool1 = nn.MaxPool2d(kernel_size=2)
            self.bn1 = nn.BatchNorm2d(32)
            self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=1)
            self.pool2 = nn.MaxPool2d(kernel_size=3)
            self.bn2 = nn.BatchNorm2d(64)
            self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
            self.pool3 = nn.MaxPool2d(kernel_size=2)
            self.bn3 = nn.BatchNorm2d(128)
            self.flatten = nn.Flatten()            
            # self.fc1 = nn.Linear(1152, 128)
            self.fc1 = nn.Linear(512, 2)
            self.relu = nn.ReLU()
            # self.sigmoid = nn.Sigmoid()

        elif embedding_model == "llama":
            self.conv1 = nn.Conv2d(1, 32, kernel_size=8, padding=1, stride = 2)
            self.pool1 = nn.MaxPool2d(kernel_size=2)
            self.bn1 = nn.BatchNorm2d(32)
            self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=1)
            self.pool2 = nn.MaxPool2d(kernel_size=3)
            self.bn2 = nn.BatchNorm2d(64)
            self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
            self.pool3 = nn.MaxPool2d(kernel_size=2)
            self.bn3 = nn.BatchNorm2d(128)
            self.flatten = nn.Flatten()            
            # self.fc1 = nn.Linear(1152, 128)
            self.fc1 = nn.Linear(512, 2)
            self.relu = nn.ReLU()
            # self.sigmoid = nn.Sigmoid()
        
        elif embedding_model == "fusion":
            self.conv1 = nn.Conv2d(1, 128, kernel_size=8, padding=1, stride = 2)
            self.pool = nn.MaxPool2d(kernel_size=2)
            self.conv2 = nn.Conv2d(128, 64, kernel_size=5, padding=1)
            self.pool = nn.MaxPool2d(kernel_size=2)
            self.conv3 = nn.Conv2d(64, 32, kernel_size=3, padding=1)
            self.pool = nn.MaxPool2d(kernel_size=2)
            self.flatten = nn.Flatten()            
            # self.fc1 = nn.Linear(96, 2)
            self.fc1 = nn.Linear(64, 2)
            # self.fc2 = nn.Linear(128, 2)
            self.relu = nn.ReLU()
            # self.sigmoid = nn.Sigmoid()
        
        elif embedding_model == "fusion3":
            self.conv1 = nn.Conv2d(1, 128, kernel_size=8, padding=1, stride = 2)
            self.pool = nn.MaxPool2d(kernel_size=2)
            self.conv2 = nn.Conv2d(128, 64, kernel_size=5, padding=1)
            self.pool = nn.MaxPool2d(kernel_size=2)
            self.conv3 = nn.Conv2d(64, 32, kernel_size=3, padding=1)
            self.pool = nn.MaxPool2d(kernel_size=2)
            self.flatten = nn.Flatten() 
            self.fc1 = nn.Linear(864, 2)
            # self.fc1 = nn.Linear(448, 2)
            self.relu = nn.ReLU()
            # self.sigmoid = nn.Sigmoid()

        elif embedding_model == "fusion4":
            self.conv1 = nn.Conv2d(1, 128, kernel_size=8, padding=1, stride = 2)
            self.pool = nn.MaxPool2d(kernel_size=2)
            self.conv2 = nn.Conv2d(128, 64, kernel_size=5, padding=1)
            self.pool = nn.MaxPool2d(kernel_size=2)
            self.conv3 = nn.Conv2d(64, 32, kernel_size=3, padding=1)
            self.pool = nn.MaxPool2d(kernel_size=2)
            self.flatten = nn.Flatten() 
            self.fc1 = nn.Linear(896, 2)
            # self.fc1 = nn.Linear(448, 2)
            self.relu = nn.ReLU()
            # self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        if self.em_model == "gpt":
            x = self.conv1(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.conv2(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.flatten(x)
            x = self.relu(self.fc1(x))
            x = self.fc2(x)
            # x = self.sigmoid(self.fc2(x))
            return x
        elif self.em_model == "mistral" or self.em_model == "flan":
            x = self.conv1(x)
            x = self.relu(x)
            x = self.pool1(x)
            x = self.bn1(x)
            x = self.conv2(x)
            x = self.relu(x)
            x = self.pool2(x)
            x = self.bn2(x)
            x = self.conv3(x)
            x = self.relu(x)
            x = self.pool3(x)
            x = self.bn3(x)
            x = self.flatten(x)
            # x = self.relu(self.fc1(x))
            x = self.fc1(x)
            # x = self.sigmoid(self.fc2(x))
            return x
        
        elif self.em_model == "llama":
            x = self.conv1(x)
            x = self.relu(x)
            x = self.pool1(x)
            x = self.bn1(x)
            x = self.conv2(x)
            x = self.relu(x)
            x = self.pool2(x)
            x = self.bn2(x)
            x = self.conv3(x)
            x = self.relu(x)
            x = self.pool3(x)
            x = self.bn3(x)
            x = self.flatten(x)
            # x = self.relu(self.fc1(x))
            x = self.fc1(x)
            # x = self.sigmoid(self.fc2(x))
            return x

        elif self.em_model == "fusion":
            x = self.conv1(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.conv2(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.conv3(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.flatten(x)
            # x = self.relu(self.fc1(x))
            x = self.fc1(x)
            # x = self.sigmoid(self.fc2(x))
            return x
        
        elif self.em_model == "fusion3":
            x = self.conv1(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.conv2(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.conv3(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.flatten(x)
            # x = self.relu(self.fc1(x))
            x = self.fc1(x)
            # x = self.sigmoid(self.fc2(x))
            return x

        elif self.em_model == "fusion4":
            x = self.conv1(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.conv2(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.conv3(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.flatten(x)
            # x = self.relu(self.fc1(x))
            x = self.fc1(x)
            # x = self.sigmoid(self.fc2(x))
            return x


In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GPT_CNN2D(embedding_model = "fusion3").to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
max_val_accuracy = 0
# Step 5: Training Loop
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).long()
        inputs = inputs.unsqueeze(1)
        optimizer.zero_grad()
        #print(inputs.shape)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
    
    # Validation
    model.eval()
    correct = 0
    total = 0
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device).long()
            inputs = inputs.unsqueeze(1)
            outputs = model(inputs)
            predicted = torch.argmax(outputs, 1)
            TP += ((predicted == 1) & (labels == 1)).sum().item()
            FP += ((predicted == 1) & (labels == 0)).sum().item()
            TN += ((predicted == 0) & (labels == 0)).sum().item()
            FN += ((predicted == 0) & (labels == 1)).sum().item()
            total += labels.size(0)
            #print(labels, predicted)
            correct += (predicted == labels).sum().item()
    val_accuracy = correct / total
    if max_val_accuracy<val_accuracy:
        max_TP = TP
        max_FP = FP
        max_TN = TN
        max_FN = FN
    max_val_accuracy = max(max_val_accuracy, val_accuracy)
    
    
    print(f'Validation Accuracy: {val_accuracy:.4f}')
    print(f'TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}')


Epoch [1/100], Loss: 0.5861
Validation Accuracy: 0.8465
TP: 350, FP: 51, TN: 422, FN: 89
Epoch [2/100], Loss: 0.2973
Validation Accuracy: 0.9441
TP: 397, FP: 9, TN: 464, FN: 42
Epoch [3/100], Loss: 0.1332
Validation Accuracy: 0.9638
TP: 431, FP: 25, TN: 448, FN: 8
Epoch [4/100], Loss: 0.0884
Validation Accuracy: 0.9507
TP: 396, FP: 2, TN: 471, FN: 43
Epoch [5/100], Loss: 0.0400
Validation Accuracy: 0.9704
TP: 432, FP: 20, TN: 453, FN: 7
Epoch [6/100], Loss: 0.0223
Validation Accuracy: 0.9682
TP: 432, FP: 22, TN: 451, FN: 7
Epoch [7/100], Loss: 0.0079
Validation Accuracy: 0.9693
TP: 431, FP: 20, TN: 453, FN: 8
Epoch [8/100], Loss: 0.0030
Validation Accuracy: 0.9704
TP: 429, FP: 17, TN: 456, FN: 10
Epoch [9/100], Loss: 0.0018
Validation Accuracy: 0.9737
TP: 427, FP: 12, TN: 461, FN: 12
Epoch [10/100], Loss: 0.0013
Validation Accuracy: 0.9748
TP: 427, FP: 11, TN: 462, FN: 12
Epoch [11/100], Loss: 0.0010
Validation Accuracy: 0.9737
TP: 426, FP: 11, TN: 462, FN: 13
Epoch [12/100], Loss: 0.0

In [12]:
print(f"Maximum Validation Accuracy:  {max_val_accuracy:.4f}")
print("TP: ", max_TP)
print("FP: ", max_FP)
print("TN: ", max_TN)
print("FN: ", max_FN)

import math

MCC = (max_TP * max_TN - max_FP * max_FN) / math.sqrt((max_TP + max_FP) * (max_TP + max_FN) * (max_TN + max_FP) * (max_TN + max_FN))

print(f'Matthews Correlation Coefficient (MCC): {MCC:.3f}')

TPR = max_TP / (max_TP + max_FN)
FPR = max_FP / (max_FP + max_TN)

print(f'True Positive Rate (TPR): {TPR:.3f}')
print(f'False Positive Rate (FPR): {FPR:.3f}')


Maximum Validation Accuracy:  0.9748
TP:  427
FP:  11
TN:  462
FN:  12
Matthews Correlation Coefficient (MCC): 0.949
True Positive Rate (TPR): 0.973
False Positive Rate (FPR): 0.023
